# Предсказание результатов матчей в Dota 2

Импортируем полезные библиотеки

In [156]:
import time
import datetime
import pandas
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

Достаем данные, разбиваем их на данные про первые 5 минут и на данные про конец игры. Пятый с конца столбец - целевая переменная.

In [157]:
raw_data_train = pandas.read_csv("features.csv")
X_train = raw_data_train.iloc[:, :-6] #early game features
y_train = raw_data_train["radiant_win"] #endgame features

raw_data_test = pandas.read_csv('features_test.csv')
X_test = raw_data_test
print(X_train[X_train.columns[X_train.isna().any()].tolist()].count())

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64


Видно, что 12 признаков для некоторых матчей могут не иметь значений. К примеру, признаки, относящиеся к времени покупки предметов (bottle_time, courier_time), могут отсутсвовать, потому что покупки не произошли в первые 5 минут. То же самое с данными о first_blood - в первые 5 минут не было first_blood, нет и данных по нему. 

У всех признаков, отвечающих за время того или иного события, пустые значения лучше заполнить большим числом. Все, что нам известно про события с отсутствующим временем - это то, что они произошли после первых 5 минут, то есть позже, чем в матчах с известными временами. Тогда эти значения должны быть больше. В случае с decision trees не так важно, насколько больше.

Есть идея в first_blood_team поменять значения с 0,1 (для первой и второй команды) на -1 и 1, а неизвестные заполнить нулями. С first_blood_player сложнее. Во-первых, это категориальный признак. Можно для него сделать one-hot encoding - но тогда будет много новых признаков, которые не очень нужны - важен не игрок, который совершил убийство, а команда. Поэтому я уберу эти признаки.

In [158]:
X_train = X_train.drop(columns=['first_blood_player1', 'first_blood_player2'])
X_train['first_blood_team'] = X_train['first_blood_team'].map(lambda x: 0 if pandas.isna(x) else x*2-1)
X_train = X_train.fillna(500)

X_test = X_test.drop(columns=['first_blood_player1', 'first_blood_player2'])
X_test['first_blood_team'] = X_test['first_blood_team'].map(lambda x: 0 if pandas.isna(x) else x*2-1)
X_test = X_test.fillna(500)


X_train.iloc[:, -24:-12]

,d5_xp,d5_gold,d5_lh,d5_kills,d5_deaths,d5_items,first_blood_time,first_blood_team,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count
0,958,1003,3,1,0,9,7.0,1.0,134.0,-80.0,244.0,2
1,1470,1622,24,0,0,9,54.0,1.0,173.0,-80.0,500.0,2
2,1350,1512,25,0,0,7,224.0,-1.0,63.0,-82.0,500.0,2
3,510,499,0,0,0,7,500.0,0.0,208.0,-75.0,500.0,0
4,1119,904,6,0,1,7,-21.0,1.0,166.0,-81.0,181.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
97225,1583,1634,10,2,0,6,114.0,-1.0,245.0,-86.0,211.0,5
97226,1791,1186,3,1,0,8,174.0,1.0,139.0,-85.0,202.0,5
97227,841,499,1,0,0,10,108.0,-1.0,43.0,-83.0,181.0,2
97228,1264,803,5,0,0,8,28.0,-1.0,96.0,-78.0,205.0,6


Теперь можно обучать классификаторы.

# Gradient Boosting approach

In [87]:
start_time = datetime.datetime.now()

random_state = 241
cv = KFold(n_splits=5, shuffle=True, random_state=random_state)
results = []
for n_estimators in [10,20,30]:    
    clf = GradientBoostingClassifier(n_estimators=n_estimators, random_state=random_state)
    results.append((np.mean(cross_val_score(clf, X_train, y_train, scoring='roc_auc')), n_estimators))
    
print('Time elapsed:', datetime.datetime.now() - start_time)


Time elapsed: 0:10:37.590439


In [88]:
results

[(0.6261886521297532, 10), (0.643284173352827, 20), (0.6507787698254051, 30)]

Выше - полученная точность при n_estimators=10, 20, 30.

Скорее всего, при увеличении n_estimators точность бы продолжала увеличиваться. Экспериментировать сложно, потому что даже на 30 обучается модель достаточно долго.

# Logistic regression approach

Сначала попробуем логистическую регресси на данных, не обрабатывая их перед этим (кроме заполнения пустых полей)

In [159]:
cv = KFold(n_splits=5, shuffle=True, random_state=random_state)
results = []
for C in [0.01, 0.1, 1, 10, 100]:    
    clf = LogisticRegression(C=C, random_state=random_state)
    results.append((np.mean(cross_val_score(clf, X_train, y_train, scoring='roc_auc')), C))


In [160]:
results

[(0.5650577070629076, 0.01),
 (0.5650577070629076, 0.1),
 (0.5650577070629076, 1),
 (0.5650577070629076, 10),
 (0.5650577070629076, 100)]

Результат не очень хороший. Что не удивительно.

Теперь удалим категориальные признаки, а так же отмасштабируем оставшиеся признаки.

In [161]:
from sklearn.preprocessing import StandardScaler

X_train_1 = X_train.drop(columns=['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', \
                               'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])
scaler = StandardScaler()
scaler.fit(X_train_1)
X_train_1 = scaler.transform(X_train_1)

In [162]:
cv = KFold(n_splits=5, shuffle=True, random_state=random_state)
results = []
for C in [0.01, 0.1, 1, 10, 100]:    
    clf = LogisticRegression(C=C, random_state=random_state)
    results.append((np.mean(cross_val_score(clf, X_train_1, y_train, scoring='roc_auc')), C))

In [163]:
results

[(0.7155935582881133, 0.01),
 (0.7156001928723871, 0.1),
 (0.715599680143528, 1),
 (0.7155997945358925, 10),
 (0.7155997902978918, 100)]

Результат заметно лучше.

Однако категориальные признаки, который мы удалили, очень важны. Разные герои в игре имеют разный процент побед - с какими-то выигрывать проще, с какими-то сложнее. Мы вернем признаки, отвечающие за выбор героев командами. Теперь если команда Radiant выбрала i-го героя, то i-е значение будет равно 1, если его выбрала команда Dire - значение будет равно -1.

In [164]:
heroes = X_train[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', \
                  'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']].values

print('maximal value for hero is', np.max(heroes))

N_heroes = np.max(heroes)
X_pick = np.zeros((heroes.shape[0], N_heroes))
for n, row in enumerate(heroes):
    for i in range(0, 5):
        X_pick[n, row[i]-1] = 1
    for i in range(5,10):
        X_pick[n, row[i]-1] = -1

X_train_2 = X_train.drop(columns=['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', \
                               'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])
X_train_2 = np.hstack((X_train_2, X_pick))
scaler = StandardScaler()
scaler.fit(X_train_2)
X_train_2 = scaler.transform(X_train_2)

maximal value for hero is 112


In [165]:
cv = KFold(n_splits=5, shuffle=True, random_state=random_state)
results = []
for C in [0.01, 0.1, 1, 10, 100]:    
    clf = LogisticRegression(C=C, random_state=random_state)
    results.append((np.mean(cross_val_score(clf, X_train_2, y_train, scoring='roc_auc')), C))

In [166]:
results

[(0.7501136136609053, 0.01),
 (0.7501137531074945, 0.1),
 (0.7501122849571419, 1),
 (0.7501120074315646, 10),
 (0.7501118930316011, 100)]

Результат стал еще лучше.

# Идеи для улучшения

1) Можно рассматривать не только выбор героев командами по отдельности, но и связки героев. В Dota некоторые герои могут хорошо между собой сочетаться - например, их способности могут быть не такими сильными по отдельности, но вместео ни очень сильны.

2) Можно использовать информацию о том, кто сделал first blood и на ком. Например, если убили более ценную позицию - это хуже, чем менее ценную. Так же, если убийство совершил герой, которому деньги и опыт за убийство тоже более важны - это тоже лучше. То есть, данная информация может быть полезной, есть смысл ее добавить, преобразовав перед этим признак из категориального.

3) Можно так же некоторые признаки, которые мы воспринимаем как числовые, трансформировать. Взять, к примеру, количество сапогов. 2 сапога на пятой минуте - намного хуже, чем 4 сапога, а не в два раза. Имеет смысл заменить количества сапогов на винрейт с ними, как вариант. 

In [169]:
clf = LogisticRegression(C=1, random_state=random_state)
clf.fit(X_train_2, y_train)
result = clf.predict_proba(X_train_2)[1]
print(sorted(result))

[0.25088322122136075, 0.7491167787786392]
